In [1]:
!pip install --quiet openai

     |████████████████████████████████| 44 kB 211 kB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 146 kB 92.4 MB/s 


In [2]:
import os
import openai
import getpass

openai.api_key = getpass.getpass(prompt='Enter OpenAI API key:')



Enter OpenAI API key:··········


## Generate paraphrases with GPT-3

In [6]:
response = openai.Completion.create(
  engine="davinci-instruct-beta",
  prompt="I am a diverse paraphrasing expert. I will generate many random pairs of original and paraphrased sentences.\n\nHere are a few:\n\noriginal: Once, a group of frogs was roaming around the forest in search of water.\nparaphrase:  A gang of frogs was roaming about the woods in search of water once more.\n\noriginal:",
  temperature=0.7,
  max_tokens=100,
  top_p=0.9,
  frequency_penalty=0.5,
  presence_penalty=0.6,
  stop=["original"]
)



In [7]:
print(response)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": " A group of frogs was roaming around the forest in search of water.\nparaphrase: A gang of frogs roamed about the woods in search of water more."
    }
  ],
  "created": 1669875911,
  "id": "cmpl-6IXINDkIonHDQgLKNgvhKiKemJYuk",
  "model": "davinci-instruct-beta",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 34,
    "prompt_tokens": 76,
    "total_tokens": 110
  }
}


In [8]:
output_string = response['choices'][0]['text'].strip()
output_modified = output_string.replace("paraphrase:","")
original_sentence, paraphrased_sentence = output_modified.split('\n')
original_sentence = original_sentence.strip()
paraphrased_sentence = paraphrased_sentence.strip()
print ("-----------------------------------------------------------------------")
print (original_sentence)
print (paraphrased_sentence)

-----------------------------------------------------------------------
A group of frogs was roaming around the forest in search of water.
A gang of frogs roamed about the woods in search of water more.


## Streamlit Dataset Creator UI

In [9]:
!pip install --quiet streamlit==1.1.0  

     |████████████████████████████████| 8.3 MB 22.7 MB/s 
     |████████████████████████████████| 182 kB 86.4 MB/s 
     |████████████████████████████████| 4.7 MB 88.9 MB/s 
     |████████████████████████████████| 78 kB 8.5 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 


In [10]:
%%writefile app.py
import pandas as pd
import streamlit as st
import json
import streamlit.components.v1 as components
import os
import openai
import getpass
import pandas as pd

# openai.api_key = getpass.getpass(prompt='Enter OpenAI API key:')
openai.api_key  = st.text_input("OpenAI API Key", '',type="password")

# st.write('# Moviepro.ai')
st.markdown("<h1 style='text-align: center; color: blue;'>Paraphrase Dataset Generator</h1>", unsafe_allow_html=True)
st.write("## Create paraphrase dataset, powered by GPT-3.")

st.write("")
if "original" not in st.session_state:
    st.session_state.original = ""
if "paraphrase" not in st.session_state:
    st.session_state.paraphrase = ""
if "df" not in st.session_state:
    st.session_state.df = pd.DataFrame(columns=['original','paraphrase'])

def getparaphrase():
  response = openai.Completion.create(
  engine="davinci-instruct-beta",
  prompt="I am a diverse paraphrasing expert. I will generate many random pairs of original and paraphrased sentences.\n\nHere are a few:\n\noriginal: Once, a group of frogs was roaming around the forest in search of water.\nparaphrase:  A gang of frogs was roaming about the woods in search of water once more.\n\noriginal:",
  temperature=0.7,
  max_tokens=100,
  top_p=0.9,
  frequency_penalty=0.5,
  presence_penalty=0.6,
  stop=["original"]
  )
  output_string = response['choices'][0]['text'].strip()
  output_modified = output_string.replace("paraphrase:","")
  original_sentence, paraphrased_sentence = output_modified.split('\n')
  st.session_state.original = original_sentence.strip()
  st.session_state.paraphrase = paraphrased_sentence.strip()

def addtodb():
  data = {
    'original': st.session_state.original,
    'paraphrase': st.session_state.paraphrase
    }

  st.session_state.df = st.session_state.df.append(data, ignore_index=True)
  st.write (st.session_state.df.shape)






st.button("Generate paraphrases",on_click=getparaphrase)
st.button("Add to Database",on_click=addtodb)

st.write("**Original:** "+st.session_state.original)
st.write("**Paraphrase:** "+st.session_state.paraphrase)
st.dataframe(st.session_state.df)

Writing app.py


In [11]:
!streamlit run app.py & npx localtunnel --port 8501

2022-12-01 06:28:07.269 INFO    numexpr.utils: NumExpr defaulting to 8 threads.

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://35.247.161.152:8501

npx: installed 22 in 3.411s
your url is: https://lovely-days-punch-35-247-161-152.loca.lt
2022-12-01 06:29:10.360 message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/davinci-instruct-beta/completions
2022-12-01 06:29:12.643 message='OpenAI API response' path=https://api.openai.com/v1/engines/davinci-instruct-beta/completions processing_ms=1552 request_id=d6f4023e855684a96990e74964b4eb6a response_code=200
2022-12-01 06:29:39.013 message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/davinci-instruct-beta/completions
2022-12-01 06:29:41.704 message='OpenAI API response' path=https://api.openai.com/v1/engines/davinci-instruct-beta/completions processing_ms=2027 request_id=608c8461dafbec5bf590247433245142 response_c